# Load Balance

The network topology consists of one switch called **s1** and three hosts, **h1**, **h2** and **h2**. Their IP and MAC addresses can be found below.

![title](imgs/load-balancing.png)

In [4]:
#!/usr/bin/python


import os
from time import sleep

from mininet.log import setLogLevel, info
from mn_wifi.cli import CLI
from mn_wifi.net import Mininet_wifi
from mn_wifi.bmv2 import ONOSBmv2Switch
from mininet.term import makeTerm


def topology():
    'Create a network.'
    net = Mininet_wifi()

    info('*** Adding hosts\n')
    h1 = net.addHost('h1', ip='10.0.0.1', mac="00:00:00:00:00:01")
    h2 = net.addHost('h2', ip='10.0.0.2', mac="00:00:00:00:00:02")
    h3 = net.addHost('h3', ip='10.0.0.3', mac="00:00:00:00:00:03")

    path = os.path.dirname(os.getcwd())
    json_file = path + '/p4-scenarios/load_balance.json'
    config = path + '/p4-scenarios/load_balance.txt'

    info('*** Adding Switch\n')
    s1 = net.addSwitch('s1', cls=ONOSBmv2Switch, netcfg=True, thriftport=50001,
                       json=json_file, switch_config=config, loglevel='debug')

    info('*** Configuring WiFi Nodes\n')
    net.configureWifiNodes()

    info('*** Creating links\n')
    net.addLink(h1, s1)
    net.addLink(h2, s1)
    net.addLink(h3, s1)

    info('*** Starting network\n')
    net.start()
    net.staticArp()

    # this is a fake mac address
    h1.cmd('arp -s 10.0.0.100 01:00:00:00:00:01')

    makeTerm(h2, title='server1', cmd="bash -c 'python load_balance_receive.py h2-eth0'")
    makeTerm(h3, title='server2', cmd="bash -c 'python load_balance_receive.py h3-eth0'")
    sleep(1)
    makeTerm(h1, title='client', cmd="bash -c 'python load_balance_send.py 10.0.0.100 "
                                     "\"P4 is cool\" h1-eth0'")

    info('*** Running CLI\n')
    CLI(net)

    os.system('pkill -f \"xterm -title\"')

    info('*** Stopping network\n')
    net.stop()


if __name__ == '__main__':
    setLogLevel('info')
    topology()


*** Adding hosts
*** Adding Switch
*** Configuring WiFi Nodes
*** Creating links
*** Starting network
*** Starting controller(s)

*** Starting L2 nodes
s1 

....⚡️ simple_switch_grpc @ 27361 thrift @ 50001



*** Running CLI
*** Starting CLI:


mininet-wifi> exit


*** Stopping network
*** Stopping 0 controllers

*** Stopping 3 links
...
*** Stopping switches/access points
s1 
*** Stopping nodes
h1 h2 h3 

*** Removing WiFi module and Configurations

*** Done


## How does it work?

table_add MyIngress.ecmp_group set_ecmp_select 10.0.0.100/32 => 0 2  
table_add MyIngress.ecmp_group set_ecmp_select 10.0.0.2/32 => 0 1  
table_add MyIngress.ecmp_group set_ecmp_select 10.0.0.3/32 => 0 1  
  
table_add MyIngress.ecmp_nhop set_nhop 0 => 00:00:00:00:00:02 10.0.0.100 2  
table_add MyIngress.ecmp_nhop set_nhop 1 => 00:00:00:00:00:03 10.0.0.100 3  